# Template Generator

## **USE CASE 1**: Referenzszenario mit ST min und Local Heat Demand ref über verschiedene Jahre und H2-Beimischungen

### Beschreibung

In diesem Use Case werden verschiedene Ausbau-Szenarien für ein Energiesystem betrachtet. Ziel ist es, die Auswirkungen unterschiedlicher Wasserstoffbeimischungen und Jahre auf das Systemverhalten zu analysieren.

### Festgelegte Annahmen

- **Solarthermie-Profil:**  
  Es wird immer das minimale Solarthermie-Profil verwendet: 
  `profiles/ST Sued_min/min_solarthermal_norm_profil_{jahr}.csv`
- **Local Heat Demand:**  
  Es wird immer das Referenzprofil genutzt:  
  `demands/local_heating/Bedarf NW-Netz/local_heat_{jahr}.csv`

### Szenarien

Für folgende Parameter werden Templates erzeugt:

- **Jahre:** 2028, 2030, 2040, 2050
- **Wasserstoffbeimischung (CHP 1 & 2 identisch):** 0%, 30%, 50%, 100%

### Namenskonvention

Die Templates werden nach folgendem Schema benannt:

`uc1_{jahr}_{admixture}h2.json`

**Beispiele:**
- `uc1_2028_0h2.json`
- `uc1_2030_30h2.json`
- `uc1_2040_50h2.json`
- `uc1_2050_100h2.json`

### Parameter im Dateinamen

- `usecase`: Nummer des Use Cases (hier immer `1`)
- `jahr`: Jahr des Szenarios (z.B. `2028`)
- `admixture`: Wasserstoffbeimischung in Prozent (z.B. `0`, `30`, `50`, `100`)

### Hinweise

- Die Wasserstoffbeimischung gilt immer für beide BHKW (CHP 1 & 2).
- Weitere Use Cases können nach dem gleichen Schema mit fortlaufender Nummerierung und ggf. zusätzlichen Parametern ergänzt werden.
- Die verwendeten Profile und Pfade sind für diesen Use Case fest definiert und werden in jedem Template entsprechend gesetzt.





In [20]:
import json
import os

# Zielordner für Templates
template_dir = r"../../config/templates"

# Feste Werte
INSTALLED_ST_POWER = 5.16  # MW
solar_thermal_profile_base = "profiles/ST Sued_min/min_solarthermal_profil_{}.csv"
normalized_solar_thermal_profile_base = "profiles/ST Sued_min/min_solarthermal_norm_profil_{}.csv"
local_heat_demand_base = "demands/local_heating/Bedarf NW-Netz/local_heat_{}.csv"

# Funktion zur Überprüfung von Schaltjahren
def is_leap_year(year):
    """Überprüft, ob ein Jahr ein Schaltjahr ist"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

# Jahre und Beimischungen
years = [2028, 2030, 2040, 2050]
admixtures = [0, 0.3, 0.5, 1]

co2_price = {
    "2028": 68.46, # EUR/t CO2
    "2030": 125.00, # EUR/t CO2
    "2040": 162.04, # EUR/t CO2
    "2050": 185.19 # EUR/t CO2
}


# for year in years:
#     # Bestimme den richtigen Heat-Demand-Pfad basierend auf Schaltjahr
#     if is_leap_year(year):
#         heat_demand_file = "demands/district_heating/default/heat_leap.csv"
#         pv_profile_suffix = "_leap"
#     else:
#         heat_demand_file = "demands/district_heating/default/heat_normal.csv"
#         pv_profile_suffix = "_normal"
    
#     # Definiere den korrekten PV-Dateipfad
#     pv_file = f"pv_capacity_factors/leipzig_t45_a180{pv_profile_suffix}.csv"
        
#     for admixture in admixtures:
#         template = {
#             "parameters": {
#                 "INSTALLED_ST_POWER": INSTALLED_ST_POWER,
#                 "HYDROGEN_ADMIXTURE_CHP_1": admixture,
#                 "HYDROGEN_ADMIXTURE_CHP_2": admixture,
#                 "CO2_PRICE": co2_price[str(year)]
#             },
#             "timeseries": {
#                 "gas_price": {
#                     "file": f"prices/ue24/gas_price_{year}.csv",
#                     "index": "t",
#                     "param": "gas_price"
#                 },
#                 "power_price": {
#                     "file": f"prices/ue24/power_price_{year}.csv",
#                     "index": "t",
#                     "param": "power_price"
#                 },
#                 "hydrogen_price": {
#                     "file": f"prices/ue24/h2_price_{year}.csv",
#                     "index": "t",
#                     "param": "hydrogen_price"
#                 },
#                 "heat_demand": {
#                     "file": heat_demand_file,
#                     "index": "t",
#                     "param": "heat_demand"
#                 },
#                 "normalized_pv_profile": {
#                     "file": pv_file,
#                     "index": "t",
#                     "param": ["timestamps", "dc_power", "normalized_pv_profile"]
#                 },
#                 "solar_thermal_heat_profile": {
#                     "file": solar_thermal_profile_base.format(year),
#                     "index": "t",
#                     "param": "solar_thermal_heat_profile"
#                 },
#                 "local_heat_demand": {
#                     "file": local_heat_demand_base.format(year),
#                     "index": "t",
#                     "param": ["local_heat_demand", "supply_temperature", "return_temperature"]
#                 },
#                 "normalized_solar_thermal_heat_profile": {
#                     "file": normalized_solar_thermal_profile_base.format(year),
#                     "index": "t",
#                     "param": "normalized_solar_thermal_heat_profile"
#                 }
#             }
#         }
#         # filename = f"uc1_{year}_{int(admixture*100)}h2.json"
#         # filepath = os.path.join(template_dir, filename)
#         # with open(filepath, "w", encoding="utf-8") as f:
#         #     json.dump(template, f, indent=2, ensure_ascii=False)
#         # print(f"Template geschrieben: {filepath} ({'Schaltjahr' if is_leap_year(year) else 'Normales Jahr'})")

---

## **USE CASE2**: Verdoppelter CO2-Preis für verschiedene H2-Beimischungen

### Beschreibung

In diesem Use Case wird untersucht, welcher CO2-Preis im Jahr 2030 notwendig wäre, damit sich ein vollständiger Wasserstoffbetrieb (100% H2) wirtschaftlich lohnt. Dazu werden unterschiedliche CO2-Preisstufen für verschiedene Wasserstoffbeimischungen simuliert und verglichen. Ziel ist es, den ökonomischen "Umschlagpunkt" zu identifizieren, ab dem die Nutzung von Wasserstoff als Brennstoff gegenüber Erdgas vorteilhaft wird

### Festgelegte Annahmen

- **Betrachtetes Jahr**: 2030 (fixiert)
- **Solarthermie-Profil**:
Es wird, wie in Use Case 1, das minimale Solarthermie-Profil verwendet:
`profiles/ST Sued_min/min_solarthermal_norm_profil_2030.csv`
- **Local Heat Demand**:
Es wird, wie in Use Case 1, das Referenzprofil genutzt:
`demands/local_heating/Bedarf NW-Netz/local_heat_2030.csv`


### Szeanrien

Für folgende Parameter werden Templates erzeugt:

- **Wasserstoffbeimischung (CHP 1 & 2 identisch):** 0%, 30%, 50%, 100%
- **CO2-Preise:** Verdoppelter Standardwert von 2030 ue24 Szenario

### Namenskonvention

`uc2_2030_{admixture}h2_2xco2.json`

### Beispiele

- `uc2_2030_0h2_2xco2.json`
- `uc2_2030_30h2_2xco2.json`
- `uc2_2030_50h2_2xco2.json`
- `uc2_2030_100h2_2xco2.json`

### Hinweise 

- Diese Analyse soll helfen, politische CO2-Preisgestaltung zur Förderung der Wasserstoffwirtschaft zu bewerten.

In [21]:
# USE CASE 2: Verdoppelter CO2-Preis (Jahr 2030)
year = 2030  # Fixiert auf 2030
admixtures = [0, 0.3, 0.5, 1]  # Wasserstoffbeimischungen wie in USE CASE 1

# Bestimme den richtigen Heat-Demand-Pfad basierend auf Schaltjahr (2030 ist kein Schaltjahr)
heat_demand_file = "demands/district_heating/default/heat_normal.csv"
pv_profile_suffix = "_normal"
pv_file = f"pv_capacity_factors/leipzig_t45_a180{pv_profile_suffix}.csv"

# Verdoppelter CO2-Preis für 2030
doubled_co2_price = co2_price["2030"] * 2

print(f"\nErstelle Templates für USE CASE 2 (Jahr 2030, CO2-Preis verdoppelt auf {doubled_co2_price} €/tCO2):")

for admixture in admixtures:
    template = {
        "parameters": {
            "INSTALLED_ST_POWER": INSTALLED_ST_POWER,
            "HYDROGEN_ADMIXTURE_CHP_1": admixture,
            "HYDROGEN_ADMIXTURE_CHP_2": admixture,
            "CO2_PRICE": doubled_co2_price  # Verdoppelter CO2-Preis
        },
        "timeseries": {
            "gas_price": {
                "file": f"prices/ue24/gas_price_{year}.csv",
                "index": "t",
                "param": "gas_price"
            },
            "power_price": {
                "file": f"prices/ue24/power_price_{year}.csv",
                "index": "t",
                "param": "power_price"
            },
            "hydrogen_price": {
                "file": f"prices/ue24/h2_price_{year}.csv",
                "index": "t",
                "param": "hydrogen_price"
            },
            "heat_demand": {
                "file": heat_demand_file,
                "index": "t",
                "param": "heat_demand"
            },
            "normalized_pv_profile": {
                "file": pv_file,
                "index": "t",
                "param": ["timestamps", "dc_power", "normalized_pv_profile"]
            },
            "solar_thermal_heat_profile": {
                "file": solar_thermal_profile_base.format(year),
                "index": "t",
                "param": "solar_thermal_heat_profile"
            },
            "local_heat_demand": {
                "file": local_heat_demand_base.format(year),
                "index": "t",
                "param": ["local_heat_demand", "supply_temperature", "return_temperature"]
            },
            "normalized_solar_thermal_heat_profile": {
                "file": normalized_solar_thermal_profile_base.format(year),
                "index": "t",
                "param": "normalized_solar_thermal_heat_profile"
            }
        }
    }
    
    # Namenskonvention gemäß Use Case 2
    filename = f"uc2_2030_{int(admixture*100)}h2_2xco2.json"
    filepath = os.path.join(template_dir, filename)
    
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(template, f, indent=2, ensure_ascii=False)
    
    print(f"Template geschrieben: {filepath}")


Erstelle Templates für USE CASE 2 (Jahr 2030, CO2-Preis verdoppelt auf 250.0 €/tCO2):
Template geschrieben: ../../config/templates\uc2_2030_0h2_2xco2.json
Template geschrieben: ../../config/templates\uc2_2030_30h2_2xco2.json
Template geschrieben: ../../config/templates\uc2_2030_50h2_2xco2.json
Template geschrieben: ../../config/templates\uc2_2030_100h2_2xco2.json


---

## Validate Templates

In [22]:
import os
import json
# Passe diese Pfade ggf. an dein Projekt an,
template_dir = os.path.abspath(os.path.join('..', '..', 'config', 'templates'))
data_dir = os.path.abspath(os.path.join('..', '..', 'data', 'input'))
missing_files = []
        
for filename in os.listdir(template_dir):
    if filename.endswith('.json'):
        template_path = os.path.join(template_dir, filename)
        with open(template_path, encoding='utf-8') as f:
                template = json.load(f)
        timeseries = template.get('timeseries', {})
        for key, entry in timeseries.items():
                rel_path = entry.get('file', None)
                if rel_path is None:
                    continue
                # Nur prüfen, wenn relativer Pfad (kein http),
                if not rel_path.startswith('http'):
                    abs_path = os.path.join(data_dir, rel_path)
                    if not os.path.exists(abs_path):
                         missing_files.append((filename, rel_path))
        
if missing_files:
    print('Fehlende Dateien:')
    for tpl, fpath in missing_files:
        print(f"Template: {tpl} → Datei fehlt: {fpath}")
else:
     print('Alle referenzierten Dateien existieren!')

Alle referenzierten Dateien existieren!
